In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
import warnings
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import (
    OneHotEncoder
)

from feature_engine.encoding import RareLabelEncoder
from sklearn.compose import ColumnTransformer
from feature_engine.datetime import DatetimeFeatures

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from utils import eda_helper_functions
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.ensemble import IsolationForest
import sklearn

from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    MinMaxScaler,
    PowerTransformer,
    FunctionTransformer
)

from feature_engine.outliers import Winsorizer
from feature_engine.datetime import DatetimeFeatures
from feature_engine.selection import SelectBySingleFeaturePerformance
from feature_engine.encoding import (
    RareLabelEncoder,
    MeanEncoder,
    CountFrequencyEncoder
)

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

import warnings
pd.set_option('display.max_columns',None)
sklearn.set_config(transform_output = "pandas")
warnings.filterwarnings("ignore")

In [3]:
dtoj_transformer = Pipeline(steps = [
    ("dt",DatetimeFeatures(features_to_extract = ['weekend'],yearfirst = True,format = "mixed"))
])

In [4]:
tuning_data = pd.read_csv("../data/val_data.csv")
X_val = tuning_data.drop(columns = ['price'])
y_val = tuning_data.price

X_val = X_val.assign(
    date = pd.to_datetime(X_val.rename(columns={'dtoj_year': 'year', 
                                     'dtoj_month': 'month', 
                                     'dtoj_day': 'day'})[['year', 'month', 'day']])
)
weekend = dtoj_transformer.fit_transform(X_val[['date']])
X_val = X_val.assign(is_weekend = weekend)
X_val.drop(columns = ['dtoj_year','dep_time_min'],inplace = True)

In [5]:
import sys
import os

# Add the parent directory (project root) to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from utils.feature_utils import (
    is_north, find_part_of_month, part_of_day,
    make_month_object, remove_duration, have_info,
    duration_category
)
from utils.rbf import RBFPercentileSimilarity,RouteCreator


In [6]:
import joblib

# Define the path to the column transformer
load_path = os.path.join("..", "artifacts", "column_transformer.joblib")

# Load the column transformer
column_transformer = joblib.load(load_path)

print("✅ column_transformer loaded successfully.")


✅ column_transformer loaded successfully.


In [7]:
X_val_transformed = column_transformer.transform(X_val)
X_val_transformed.head(5)

,tf1__airline_Air India,tf1__airline_Indigo,tf1__airline_Jet Airways,tf1__airline_Multiple Carriers,tf1__airline_Other,tf2__date_weekend,tf3__route,tf3__source_is_north,tf3__destination_is_north,tf4__part_of_month,tf4__dtoj_day,tf5__part_of_day,tf5__dep_time_hour,tf6__pca0,tf6__dtoj_month,tf7__pca0,tf7__total_stops,tf7__is_direct_flight,tf8__duration_rbf_25,tf8__duration_rbf_50,tf8__duration_rbf_75,tf8__duration_cat,tf8__duration,tf9__additional_info_Other,tf9__additional_info_in-flight meal not included,tf9__additional_info_no info
0,0.0,0.0,1.0,0.0,0.0,0,10497.166667,1,0,-1.035975,1.000000,4,0.391304,-0.300968,-0.031323,2.710959,2,0,-0.361153,-0.100935,-0.082143,2.0,2.031084,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0,10497.166667,1,0,-1.035975,1.000000,16,0.739130,-0.321764,-0.212830,-0.209998,1,0,-0.361153,-0.100935,-0.082143,1.0,-0.609718,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0,7991.070822,0,1,-0.669521,0.538462,20,0.956522,-0.259376,1.249597,0.089383,1,0,-0.361153,-0.100935,-0.082143,2.0,-0.181480,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1,9146.543996,0,0,-0.669521,0.653846,12,0.608696,0.697084,-0.031323,-1.527130,0,1,-0.361153,-0.100935,-0.082143,0.0,-0.956387,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0,4905.245940,0,0,-1.035975,0.884615,16,0.826087,-0.321764,-0.212830,-1.612668,0,1,-0.361153,-0.100935,-0.082143,0.0,-1.078741,0.0,0.0,1.0


In [8]:
X_train_transformed = pd.read_csv("../transformed_data/X_train_transformed.csv")
X_test_transformed = pd.read_csv("../transformed_data/X_test_transformed.csv")

y_train = pd.read_csv("../data/train_data.csv").price
y_test = pd.read_csv("../data/test_data.csv").price

### Model Tuning

In [9]:
# Taking the XGBoost Model 

import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import joblib  # for saving model

def objective(trial):
    # Suggest hyperparameters
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 10.0),
        "random_state": 42,
        "n_jobs": -1
    }

    # Model
    model = XGBRegressor(**param)

    # Fit on training data
    model.fit(X_train_transformed, y_train)

    # Predict on tuning set
    preds = model.predict(X_val_transformed)

    # Evaluation metric
    mse = mean_squared_error(y_val, preds)
    return mse  # Minimize MSE


In [10]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100) 

[I 2025-08-26 19:09:44,019] A new study created in memory with name: no-name-190ea420-0ab1-4e33-b5a0-abd0b8c370b3


[I 2025-08-26 19:09:55,720] Trial 0 finished with value: 2588726.75 and parameters: {'n_estimators': 815, 'max_depth': 14, 'learning_rate': 0.15421895183668394, 'subsample': 0.5938212727380714, 'colsample_bytree': 0.9447542534244757, 'gamma': 1.3264370013985471, 'min_child_weight': 1, 'reg_alpha': 1.1302875873693163, 'reg_lambda': 9.951116381652241}. Best is trial 0 with value: 2588726.75.
[I 2025-08-26 19:10:01,197] Trial 1 finished with value: 2297637.25 and parameters: {'n_estimators': 947, 'max_depth': 19, 'learning_rate': 0.057319164996236356, 'subsample': 0.6094711372718187, 'colsample_bytree': 0.5327346366981491, 'gamma': 6.544856663568152, 'min_child_weight': 7, 'reg_alpha': 6.492841442811031, 'reg_lambda': 9.755442986249342}. Best is trial 1 with value: 2297637.25.
[I 2025-08-26 19:10:05,253] Trial 2 finished with value: 2448328.75 and parameters: {'n_estimators': 627, 'max_depth': 19, 'learning_rate': 0.10644247023594954, 'subsample': 0.7541729230887094, 'colsample_bytree': 0

In [11]:
best_params = study.best_params
best_params["random_state"] = 42
best_params["n_jobs"] = -1

final_model = XGBRegressor(**best_params)
final_model.fit(X_train_transformed, y_train)

model_dir = os.path.join("..", "artifacts", "models")
os.makedirs(model_dir, exist_ok=True)

# Full save path for model
model_path = os.path.join(model_dir, "xgb_flight_price_model.joblib")

# Save the model
joblib.dump(final_model, model_path)

print(f"✅ Final model trained and saved as {model_path}")

✅ Final model trained and saved as ..\artifacts\models\xgb_flight_price_model.joblib


### Loading the model

In [12]:
import joblib
models_path = os.path.join("..", "artifacts", "models")
xgboostmodel_path = os.path.join(models_path,"xgb_flight_price_model.joblib")
final_model = joblib.load(xgboostmodel_path)
print("✅ Model loaded successfully.")

✅ Model loaded successfully.


### Checking performance on Test data 

In [13]:
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error

y_test_pred = final_model.predict(X_test_transformed)

print("Test MSE:", mean_squared_error(y_test, y_test_pred))
print("Test MAE:", mean_absolute_error(y_test, y_test_pred))
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_test_pred)))
print("Test R²:", r2_score(y_test, y_test_pred))

Test MSE: 3122815.0
Test MAE: 894.5504150390625
Test RMSE: 1767.1488335734487
Test R²: 0.8581041097640991


### Testing if the model is overfitting or not

In [14]:
# First on training data

y_train_pred = final_model.predict(X_train_transformed)

print("Train MSE:", mean_squared_error(y_train, y_train_pred))
print("Train MAE:", mean_absolute_error(y_train, y_train_pred))
print("Train RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)))
print("Train R²:", r2_score(y_train, y_train_pred))

Train MSE: 1108041.625
Train MAE: 626.0498657226562
Train RMSE: 1052.635561341151
Train R²: 0.9493624567985535


In [15]:
# On validation data

y_val_pred = final_model.predict(X_val_transformed)

print("Val MSE:", mean_squared_error(y_val, y_val_pred))
print("Val MAE:", mean_absolute_error(y_val, y_val_pred))
print("Val RMSE:", np.sqrt(mean_squared_error(y_val, y_val_pred)))
print("Val R²:", r2_score(y_val, y_val_pred))

Val MSE: 2023584.25
Val MAE: 826.5821533203125
Val RMSE: 1422.5274162560102
Val R²: 0.8913947343826294


In [16]:
cleaned_flight_data = pd.read_csv("../data/cleaned_flight_data.csv")

In [17]:
cleaned_flight_data["date"] = pd.to_datetime(cleaned_flight_data.rename(columns={'dtoj_year': 'year', 
                                     'dtoj_month': 'month', 
                                     'dtoj_day': 'day'})[['year', 'month', 'day']])
weekend = dtoj_transformer.fit_transform(cleaned_flight_data[['date']])
cleaned_flight_data = cleaned_flight_data.assign(is_weekend = weekend)
cleaned_flight_data.drop(columns = ['dep_time_min','dtoj_year'],inplace = True)

X = cleaned_flight_data.drop(columns = ['price'])
y = cleaned_flight_data.price

In [18]:
cleaned_flight_data.head(2)

,airline,source,destination,duration,total_stops,additional_info,price,dep_time_hour,dtoj_day,dtoj_month,date,is_weekend
0,Indigo,banglore,new delhi,170,0,no info,3897,22,24,3,2019-03-24,1
1,Air India,kolkata,banglore,445,2,no info,7662,5,1,5,2019-05-01,0


In [19]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline

# Suppose you already have:
# column_transformer (your preprocessing steps)
# best_model (your tuned ML model, e.g. RandomForest with best params)

# Build pipeline = preprocessing + tuned model
final_pipeline = Pipeline(steps=[
    ('preprocess', column_transformer),
    ('model', final_model)
])

# Define KFold
kfold = KFold(n_splits=5, shuffle=True)

# Run CV
scores = cross_val_score(
    final_pipeline, 
    X,  # full cleaned raw data (without manual splitting)
    y, 
    cv=kfold, 
    scoring='r2'
)

print("Cross-validation R² scores:", scores)
print("Mean R²:", scores.mean())
print("Std Dev:", scores.std())


Cross-validation R² scores: [0.8722229  0.82386392 0.87094951 0.84427643 0.90207869]
Mean R²: 0.8626782894134521
Std Dev: 0.026672411019507767


In [20]:
# Suppose y_true and y_pred are arrays
df = pd.DataFrame({"y_true": y_test, "y_pred": y_test_pred})
df["error"] = abs(df["y_true"] - df["y_pred"])
df["perc_error"] = df["error"] / df["y_true"]

# Price bins (quartiles or custom ranges)
bins = [0, 5000, 10000, 20000, 80000]
labels = ["Budget (<5k)", "Mid (5k-10k)", "Upper-mid (10k-20k)", "Premium (20k+)"]
df["price_bin"] = pd.cut(df["y_true"], bins=bins, labels=labels)

# Aggregate error by bin
df.groupby("price_bin")[["error", "perc_error"]].mean()

,error,perc_error
price_bin,,
Budget (<5k),445.633206,0.115334
Mid (5k-10k),892.914654,0.124541
Upper-mid (10k-20k),963.724430,0.074382
Premium (20k+),6437.664779,0.193675


In [21]:
print("Total rows below 10000 are",y_train.to_frame().query('price < 10000').shape[0])
print("Total rows between 10000 to 20000 are",y_train.to_frame().query('price < 20000 & price > 10000').shape[0])
print("Total rows above 20000 are",y_train.to_frame().query('price > 20000').shape[0])
print("Total rows are" ,y_train.shape[0])

Total rows below 10000 are 4013
Total rows between 10000 to 20000 are 2581
Total rows above 20000 are 100
Total rows are 6694


#### Applying the MOE concept

In [ ]:
train_df = X_train_transformed.copy()
val_df = X_val_transformed.copy()
train_df['price'] = y_train.values 
val_df['price'] = y_val.values
train_df_low = train_df.query('price < 10000')
train_df_high = train_df.query('price >10000 & price < 20000')
X_train_low = train_df_low.drop(columns=['price'])
y_train_low = train_df_low[['price']]
X_train_high = train_df_high.drop(columns=['price'])
y_train_high = train_df_high[['price']]

val_df_low = val_df.query('price < 10000')
val_df_high = val_df.query('price >10000 & price < 20000') 
X_val_low = val_df_low.drop(columns=['price'])
y_val_low = val_df_low[['price']]
X_val_high = val_df_high.drop(columns=['price'])
y_val_high = val_df_high[['price']]

In [48]:
# Tuning the Expert_1 model

import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import joblib  # for saving model

def objective(trial):
    # Suggest hyperparameters
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 10.0),
        "random_state": 42,
        "n_jobs": -1
    }

    # Model
    model = XGBRegressor(**param)

    # Fit on training data
    model.fit(X_train_low, y_train_low)

    # Predict on tuning set
    preds = model.predict(X_val_low)

    # Evaluation metric
    mse = mean_squared_error(y_val_low, preds)
    return mse  # Minimize MSE


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50) 

[I 2025-08-22 17:56:03,122] A new study created in memory with name: no-name-68c2ba71-fcc1-4af2-9d20-0f7dca2771a8
[I 2025-08-22 17:56:05,409] Trial 0 finished with value: 463833.34375 and parameters: {'n_estimators': 930, 'max_depth': 7, 'learning_rate': 0.20928911605860895, 'subsample': 0.8146156519474925, 'colsample_bytree': 0.5555763318241056, 'gamma': 3.3201172517636723, 'min_child_weight': 8, 'reg_alpha': 1.3613028007805095, 'reg_lambda': 9.599296893583173}. Best is trial 0 with value: 463833.34375.
[I 2025-08-22 17:56:06,829] Trial 1 finished with value: 462390.15625 and parameters: {'n_estimators': 218, 'max_depth': 19, 'learning_rate': 0.19296265846924185, 'subsample': 0.6897864642282869, 'colsample_bytree': 0.6737867702585008, 'gamma': 4.297854844214793, 'min_child_weight': 5, 'reg_alpha': 8.879478500341135, 'reg_lambda': 5.030780348999311}. Best is trial 1 with value: 462390.15625.
[I 2025-08-22 17:56:08,466] Trial 2 finished with value: 424394.8125 and parameters: {'n_estima

In [49]:
# Expert_model_1

best_params = study.best_params
best_params["random_state"] = 42
best_params["n_jobs"] = -1

expert_model_1 = XGBRegressor(**best_params)
expert_model_1.fit(X_train_low, y_train_low)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.64362331491217, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=2.4437833689018267, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.017295415039278283,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=542, n_jobs=-1,
             num_parallel_tree=None, ...)

In [50]:
# Tuning the Expert_2 model

import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import joblib  # for saving model

def objective(trial):
    # Suggest hyperparameters
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 10.0),
        "random_state": 42,
        "n_jobs": -1
    }

    # Model
    model = XGBRegressor(**param)

    # Fit on training data
    model.fit(X_train_high, y_train_high)

    # Predict on tuning set
    preds = model.predict(X_val_high)

    # Evaluation metric
    mse = mean_squared_error(y_val_high, preds)
    return mse  # Minimize MSE


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50) 

[I 2025-08-22 17:58:19,008] A new study created in memory with name: no-name-f7c1ca2e-c2b6-4c12-bafc-d4c263868e11
[I 2025-08-22 17:58:19,716] Trial 0 finished with value: 1147825.875 and parameters: {'n_estimators': 827, 'max_depth': 3, 'learning_rate': 0.1621554359513038, 'subsample': 0.9494709983235883, 'colsample_bytree': 0.5790007366865471, 'gamma': 5.4316161764530415, 'min_child_weight': 9, 'reg_alpha': 8.40703808337874, 'reg_lambda': 3.094946541572895}. Best is trial 0 with value: 1147825.875.
[I 2025-08-22 17:58:22,842] Trial 1 finished with value: 994775.75 and parameters: {'n_estimators': 887, 'max_depth': 14, 'learning_rate': 0.05332368423250739, 'subsample': 0.7488496020348023, 'colsample_bytree': 0.5379916607111718, 'gamma': 8.65769826827566, 'min_child_weight': 10, 'reg_alpha': 7.90975861418498, 'reg_lambda': 9.198223777190158}. Best is trial 1 with value: 994775.75.
[I 2025-08-22 17:58:23,554] Trial 2 finished with value: 1024238.625 and parameters: {'n_estimators': 843, 

In [51]:
# Expert_model_2

best_params = study.best_params
best_params["random_state"] = 42
best_params["n_jobs"] = -1

expert_model_2 = XGBRegressor(**best_params)
expert_model_2.fit(X_train_high, y_train_high)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9238570961008844, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=1.6547654447638713, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.021773672548885882,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=12, max_leaves=None,
             min_child_weight=10, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=598, n_jobs=-1,
             num_parallel_tree=None, ...)

In [52]:
# Making the gate 

from xgboost import XGBClassifier

y_cluster = (y_train < 10000).astype(int)  # 1 if y_train<10k, else 0

gating = XGBClassifier(n_estimators=200, max_depth=4)
gating.fit(X_train_transformed, y_cluster)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, ...)

In [53]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Predict on validation / test
y_gate_pred = gating.predict(X_train_transformed)

# Metrics
print("Accuracy:", accuracy_score(y_cluster, y_gate_pred))
print("Precision:", precision_score(y_cluster, y_gate_pred))
print("Recall:", recall_score(y_cluster, y_gate_pred))
print("F1:", f1_score(y_cluster, y_gate_pred))

Accuracy: 0.9708694353152076
Precision: 0.9732275656916212
Recall: 0.9783204585098431
F1: 0.975767366720517


In [54]:
import numpy as np

def moe_predict(X):
    # step 1: gating probs
    probs = gating.predict_proba(X)  # shape (n_samples, 2)
    p_low = probs[:, 1]   # prob cluster=low
    p_high = probs[:, 0]  # prob cluster=high
    
    # step 2: expert predictions
    pred_low = expert_model_1.predict(X)
    pred_high = expert_model_2.predict(X)
    
    # step 3: weighted combination
    pred_final = p_low * pred_low + p_high * pred_high

    return pred_final


In [55]:
y_test_pred_moe = moe_predict(X_test_transformed)
rmse_moe = np.sqrt(mean_squared_error(y_test, y_test_pred_moe))
mae_moe = mean_absolute_error(y_test, y_test_pred_moe)
r2_moe = r2_score(y_test, y_test_pred_moe)
print("Rmse score",rmse_moe)
print("mae score",mae_moe)
print("R2 score",r2_moe)

Rmse score 2153.93639646114
mae score 887.8986206054688
R2 score 0.7891908884048462


In [56]:
df_moe = pd.DataFrame({
    "y_true": y_test,
    "y_pred": y_test_pred_moe
})

# 3. Compute absolute + percentage error
df_moe["error"] = abs(df_moe["y_true"] - df_moe["y_pred"])
df_moe["perc_error"] = df_moe["error"] / df_moe["y_true"]

# 4. Define bins
bins = [0, 5000, 10000, 20000, 80000]
labels = ["Budget (<5k)", "Mid (5k-10k)", "Upper-mid (10k-20k)", "Premium (20k+)"]

df_moe["price_bin"] = pd.cut(df_moe["y_true"], bins=bins, labels=labels)

# 5. Aggregate error by bin
df_moe.groupby("price_bin")[["error", "perc_error"]].mean()



,error,perc_error
price_bin,,
Budget (<5k),348.403659,0.091302
Mid (5k-10k),785.053211,0.109262
Upper-mid (10k-20k),897.606056,0.068109
Premium (20k+),12093.611165,0.388537


### Taking the Random Forest model

In [20]:
# Taking the Random Forest model

import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def objective(trial):
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 5, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),  # ✅ fixed
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "random_state": 42,
        "n_jobs": -1
    }

    model = RandomForestRegressor(**param)
    model.fit(X_train_transformed, y_train)

    preds = model.predict(X_val_transformed)
    mse = mean_squared_error(y_val, preds)

    return mse

# Run the study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)


[I 2025-08-22 17:34:30,960] A new study created in memory with name: no-name-d582bb2e-f07f-4473-a974-2ed9575b893a
[I 2025-08-22 17:34:32,420] Trial 0 finished with value: 3949740.988178595 and parameters: {'n_estimators': 332, 'max_depth': 9, 'min_samples_split': 5, 'min_samples_leaf': 6, 'max_features': 'log2', 'bootstrap': True}. Best is trial 0 with value: 3949740.988178595.
[I 2025-08-22 17:34:35,092] Trial 1 finished with value: 3235487.863368827 and parameters: {'n_estimators': 848, 'max_depth': 17, 'min_samples_split': 15, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'bootstrap': True}. Best is trial 1 with value: 3235487.863368827.
[I 2025-08-22 17:34:36,873] Trial 2 finished with value: 3487995.696783186 and parameters: {'n_estimators': 602, 'max_depth': 18, 'min_samples_split': 13, 'min_samples_leaf': 8, 'max_features': 'sqrt', 'bootstrap': True}. Best is trial 1 with value: 3235487.863368827.
[I 2025-08-22 17:34:38,152] Trial 3 finished with value: 2945370.4606393795 and p

In [21]:
best_params = study.best_params
best_params["random_state"] = 42
best_params["n_jobs"] = -1

final_rf_model = RandomForestRegressor(**best_params)
final_rf_model.fit(X_train_transformed, y_train)

model_dir = os.path.join("..", "artifacts", "models")
os.makedirs(model_dir, exist_ok=True)

# Full save path for model
model_path = os.path.join(model_dir, "randomforest_flight_price_model.joblib")

# Save the model
joblib.dump(final_rf_model, model_path)

print(f"✅ Final model trained and saved as {model_path}")

✅ Final model trained and saved as ..\artifacts\models\randomforest_flight_price_model.joblib


In [22]:
from sklearn.metrics import r2_score, mean_absolute_error

y_pred = final_rf_model.predict(X_test_transformed)

print("Test MSE:", mean_squared_error(y_test, y_pred))
print("Test MAE:", mean_absolute_error(y_test, y_pred))
print("Test R²:", r2_score(y_test, y_pred))

Test MSE: 3620046.3503122544
Test MAE: 974.8438947050673
Test R²: 0.8355106881602943


In [23]:
y_pred = final_rf_model.predict(X_train_transformed)

print("Train MSE:", mean_squared_error(y_train, y_pred))
print("Train MAE:", mean_absolute_error(y_train, y_pred))
print("Train R²:", r2_score(y_train, y_pred))

Train MSE: 1488026.9397696452
Train MAE: 629.9009802370945
Train R²: 0.93199710266873
